# Preamble

In [32]:
from scipy.spatial.distance import cdist
rng = np.random.default_rng()
X = rng.integers(low = 0, high = 10, size=(3,2))
print(X)
cdist(X,X)

[[6 1]
 [3 3]
 [2 3]]


array([[0.               , 3.605551275463989, 4.47213595499958 ],
       [3.605551275463989, 0.               , 1.               ],
       [4.47213595499958 , 1.               , 0.               ]])

In [15]:
n = 20
m = 100 
k = 200

X = np.random.randn(m,2)

Y = np.random.randn(k,n)


X = np.random.Generator.integers(low = 0, high = 100, size = (50,2))

def scipy_cdist(X,Y):
  return cdist(X,Y,metric='euclidean')

def numpy_for_loop(X,Y):
    m = X.shape[0]
    k = Y.shape[0]
    D = np.zeros((m,k))
    for i in range(m):
        for j in range(k):
            D[i,j] = np.sum((X[i] - Y[j])**2)**0.5
    return D

def numpy_vectorized(X,Y):
    return np.sum((X[:,None,:] - Y[None,:,:])**2, axis=2)**0.5
print(X)
scipy_cdist(X,X)

TypeError: unbound method Generator.integers() needs an argument

In [8]:
# load packages
import numpy as np
np.set_printoptions(precision=20)
import arcpy
import os
import scipy as scipy
from scipy.linalg import eig
import pandas as pd
pd.set_option('display.max_columns', None)
import networkx as nx
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'arcpy'

# Helper functions

In [35]:
# Helper functions

def create_or_get_gdb(out_folder_path, gdb_name):
    # Check if the geodatabase already exists
    gdb_path = os.path.join(out_folder_path, gdb_name)
    if arcpy.Exists(gdb_path):
        print(f"Geodatabase '{gdb_name}' already exists at {gdb_path}")
        return gdb_path
    else:
        # Create the geodatabase
        arcpy.CreateFileGDB_management(out_folder_path, gdb_name)
        print(f"Geodatabase '{gdb_name}' created at {gdb_path}")
        return gdb_path    
# Function to calculate distances, both Euclidian and "realised"/"Experienced" distances, where the latter is a probability output
def calculate_distance_matrix(input_feature_class, Alpha = 1/1000, ExportLines = False):
    """
    This function calculates a distance matrix for all points in the input feature class using NumPy arrays.
    
    :param input_feature_class: Path to the input feature class containing point geometries.
    :return: A NumPy array representing the distance matrix.
    """
    # Retrieve the spatial reference of the input feature class.
    sr = arcpy.Describe(input_feature_class).spatialReference

    # Get the number of points in the input feature class.
    point_count = int(arcpy.GetCount_management(input_feature_class).getOutput(0))

    # Initialize a NumPy array to store the distance matrix with zeros.
    distance_matrix = np.zeros((point_count, point_count))
    # Initialize a NumPy array to store the dispersal probabilities.
    distance_matrix_exp = np.zeros((point_count, point_count))

    # Create a list to store geometries and their IDs.
    geometries = []    
    
    # Here we have chunk where we create a feature class where we can store the lines going btw all the features
    # Define the spatial reference
    spatial_ref = arcpy.Describe(input_feature_class).spatialReference
    
    # Use a search cursor to iterate through the features and store geometries.
    with arcpy.da.SearchCursor(input_feature_class, ["OID@", "SHAPE@"], spatial_reference=sr) as cursor:
        for row in cursor:
            oid, geom = row
            geometries.append((oid, geom))

    # Calculate distances and fill the NumPy array.
    for i, (oid_from, geom_from) in enumerate(geometries):
        #print(i)
        for j, (oid_to, geom_to) in enumerate(geometries):
            if oid_from != oid_to:  # Ensure that we don't calculate the distance to the same feature.
                distance = geom_from.distanceTo(geom_to)  # Calculate the distance to another feature.
                distance_matrix[i, j] = distance  # Store the calculated distance in the array.
                distance_matrix_exp[i, j] = np.exp(-distance*Alpha)  # Store the calculated distance in the array.
                #if ExportLines is True
                    # Create a new feature class
                    #new_feature_class = 'Paths'
                    #arcpy.CreateFeatureclass_management(arcpy.env.workspace, new_feature_class, 'POLYLINE', spatial_reference=spatial_ref)
                    #start_point = geom_from.centroid # now we move over to generating lines btw all the geometries
                #    end_point = geom_to.centroid  #  this lines are later used to illustrate sensitities of the connections
                #    line = arcpy.Polyline(arcpy.Array([start_point, end_point]), spatial_ref) # create a line
                #    # TO-DO: Add OID
                #    # Use an insert cursor to add the new line to the feature class
                #    with arcpy.da.InsertCursor(new_feature_class, ['SHAPE@']) as cursor:
                #        cursor.insertRow([line])

    return distance_matrix, distance_matrix_exp  # Return the complete distance matrix.

def calculate_areas_and_outer_matrix(feature_class):
    """
    Calculate the areas of features in a feature class and the outer product matrix of these areas.
    
    :param feature_class: Path to the input feature class.
    :return: A tuple containing a list of areas and the outer product matrix.
    """
    # Step 1: Extract Areas
    areas = []
    with arcpy.da.SearchCursor(feature_class, ["SHAPE@AREA"]) as cursor:
        for row in cursor:
            areas.append(row[0])  # Append the area of the feature to the list

    # Step 2: Create Outer Matrix
    areas_array = np.array(areas)  # Convert the list of areas to a NumPy array
    outer_matrix = np.outer(areas_array, areas_array)  # Calculate the outer product matrix

    return areas, outer_matrix

In [36]:
# Main function
def CalcEigenmeasures(LandscapeMatrix):
    # Define the function to calculate contribution
    def calculate_contribution(Lambda, Mij):
        num_patches = Mij.shape[0]
        contributions = np.zeros(num_patches)
        
        for ii in range(num_patches):
            # Exclude the ii-th row and column from Mij
            Mij_sub = np.delete(Mij, ii, axis=0)
            Mij_sub = np.delete(Mij_sub, ii, axis=1)
            
            # Calculate the largest eigenvalue of the modified Mij
            eigenvalues = np.linalg.eigvals(Mij_sub)
            largest_eigenvalue = np.max(eigenvalues)
            
            # Calculate the contribution
            contributions[ii] = (Lambda - largest_eigenvalue) / Lambda
        
        return contributions

    eigenvalues, eigenvectors = scipy.linalg.eig(Mij)
    Le = np.argmax(eigenvalues)
    Lambda = np.real(eigenvalues[Le])
    
    # Approximate stable age distribution = right eigenvector
    w0 = np.real(eigenvectors[:, Le])
    w = np.abs(w0)
    # Reproductive value = left eigenvector
    # This equals the right eigenvector of the landscape matrix transposed
    V = np.linalg.inv(eigenvectors).conj()
    v = np.abs(np.real(V[Le, ]))
    
    # Contribution of the patch to lambda
    # When considering small perturbations, loses a small part of the habitats and habitat degradation
    pc = v.T * w# # These are normalized
    #pc_small = pc / np.sum(pc)  # Normalize the contributions

    # 'pc_s' now contains the normalized contribution of each patch to lambda
    # Call the function with your Lambda and Mij data
    contributions = calculate_contribution(Lambda, Mij)

    # Collect all the measures
    EigenMeasures = pd.DataFrame({"NiNID":ids,
                                  "PC_small":pc,
                                  "PC_large":contributions,
                                  "REv":w, 
                                  "LEv":v}, index = names)
    return EigenMeasures


def communities(landscape_matrix):
    # Convert the input matrix to a NumPy array
    M = np.array(landscape_matrix)
    # Calculate the total number of edges/links in the landscape
    m = np.sum(M > 0)
    
    # Calculate the degrees of the vertices/nodes/patches
    kj = np.sum(M > 0, axis=0)
    ki = np.sum(M > 0, axis=1)
    
    # Define the modularity matrix
    B = M - np.outer(ki, kj) / (2 * m)
    # Compute the eigenvalues and eigenvectors of the modularity matrix
    ev = np.linalg.eig(B)
    
    # Find the index of the largest eigenvalue
    lmax = np.argmax(np.real(ev[0]))
    
    # Get the eigenvector corresponding to the largest eigenvalue
    W = ev[1][:, lmax]
    
    # Create a frame with indices and group labels based on the sign of the eigenvector
    w_frame = np.column_stack((np.arange(len(W)), np.array([chr(97 + int(x)) for x in np.sign(W) + 1])))
    # Generate positions for the nodes for visualization
    #pos = nx.spring_layout(nx.from_numpy_matrix(M))
    
    # Plot the nodes with colors based on their group labels
    #plt.scatter(*zip(*pos.values()), c=[ord(x) for x in w_frame[:, 1]])
    #plt.show()
    print("Run loop")
    t = 0
    while len(np.unique(w_frame[:, t])) != len(np.unique(w_frame[:, t + 1])):
        print(t)
        new_col = np.empty(len(w_frame), dtype=object)
        
        for i in np.unique(w_frame[:, t + 1]):
            idx = np.where(w_frame[:, t + 1] == i)[0]
            subB = B[np.ix_(idx, idx)]
            ev = np.linalg.eig(subB)
            lmax = np.argmax(np.real(ev[0]))
            W = ev[1][:, lmax]
            new_col[idx] = [f"{w_frame[idx[0], t + 1]}-{chr(97 + int(x))}" for x in np.sign(W) + 1]
        #print(new_col)
        w_frame = np.column_stack((w_frame, new_col))
        #w_frame[:, t + 1] = np.array([ord(x) for x in w_frame[:, t + 1]])
        # Convert the first column of characters to numbers starting from 1
        unique_chars, indices = np.unique(w_frame[:, t + 1], return_inverse=True)
        w_frame[:, t + 1] = indices + 1
        t += 1
    
    return w_frame[:, :-1]


##    Bruk

In [37]:
# Example usage of the function.
# Set the path where you want to create the file geodatabase
out_folder_path = "C:/Users/endofs/OneDrive - Miljødirektoratet/Kartfiler/Nettverk"
# Set the name for your new file geodatabase
gdb_name = "Nettverk.gdb"

# File to work with
input_fc = "C:/Users/endofs/OneDrive - Miljødirektoratet/Dokumenter/ArcGIS/Projects/MyProject/MyProject.gdb/GRUK"


# Set the workspace environment to your desired directory
arcpy.env.workspace = out_folder_path
# Enable overwriting of output
arcpy.env.overwriteOutput = True
gdb_path = create_or_get_gdb(out_folder_path, gdb_name)

Distances = calculate_distance_matrix(input_fc, Alpha = 1/1500, ExportLines = False)  # Call the function and store the result.
#print(result[0])
#print(result[1])
areas = calculate_areas_and_outer_matrix(input_fc)
Mij = Distances[1] * areas[1]
# Get the number of columns in the ndarray to determine the size
array_size = Mij.shape[1]
# Generate 'names' and 'formats' based on the array size
names = []
ids = []
with arcpy.da.SearchCursor(input_fc, ["OID@","NiNID"]) as cursor:
    for row in cursor:
        names.append(str(row[0]))
        ids.append(str(row[1]))

formats = ['f8'] * array_size  # Assuming all fields are of type float ('f8')
# Create a compound dtype using the names and formats
dtype = np.dtype({'names': names, 'formats': formats})
# Convert the ndarray to a structured array
Mij_str = np.core.records.fromarrays(Mij.transpose(), dtype=dtype)
# Enable overwriting of output
arcpy.env.overwriteOutput = True
# Use the Table To Geodatabase tool to import the CSV into the file geodatabase
table_path = out_folder_path+"/"+gdb_name+"/LandscapeMatrix"
if arcpy.Exists(table_path):
    arcpy.Delete_management(table_path)
arcpy.da.NumPyArrayToTable(Mij_str, table_path)
print("Ferdig")

Geodatabase 'Nettverk.gdb' already exists at C:/Users/endofs/OneDrive - Miljødirektoratet/Kartfiler/Nettverk\Nettverk.gdb
Ferdig


In [38]:
EigenMeasures = CalcEigenmeasures(Mij)
EigenMeasures["Area"] = areas[0]
print(EigenMeasures.head)


[36]:18: ComplexWarning: Casting complex values to real discards the imaginary part


<bound method NDFrame.head of                NiNID      PC_small      PC_large           REv           LEv  \
1    NINFP2110013705  4.997741e-24  1.234119e-15  2.235468e-12  2.235658e-12   
2    NINFP2010006340  1.169922e-23 -3.173449e-15  3.420397e-12  3.420427e-12   
3    NINFP2110029289  1.526052e-24  0.000000e+00  1.235333e-12  1.235337e-12   
4    NINFP2010014883  2.101455e-34 -3.349752e-15  3.744973e-17  5.611402e-18   
5    NINFP1810016775  1.790253e-35 -1.586725e-15  1.850446e-17  9.674714e-19   
..               ...           ...           ...           ...           ...   
456  NINFP2110017405  1.091950e-24  5.289082e-16  1.044940e-12  1.044988e-12   
457  NINFP2010047611  2.764291e-25 -8.815136e-16  5.257722e-13  5.257584e-13   
458  NINFP2110069698  2.764357e-25 -8.815136e-16  5.257722e-13  5.257709e-13   
459  NINFP2010047823  5.064657e-24 -2.468238e-15  2.250495e-12  2.250464e-12   
460  NINFP2110062376  5.064651e-24 -2.468238e-15  2.250495e-12  2.250461e-12   

         

## Finding communities

In [39]:
Coms = communities(Mij)
print("Ferdig")

Run loop
0
1
2
3
4
5
6


[36]:93: ComplexWarning: Casting complex values to real discards the imaginary part


7
8
9
10
11
12
13
14
15
Ferdig


In [64]:
# Function to bin scores within each group or for the entire DataFrame
def bin_scores(df, column='PC_large', group=None, cuts=None, labels=None):
    if cuts is None:
        cuts = np.quantile(df[column], [1/3, 2/3])
    if labels is None:
        labels = ['Lav', 'Middels', 'Høy']
    
    def bin_group(group_df):
        group_cuts = np.quantile(group_df[column], [1/3, 2/3]) if cuts is None else cuts
        group_df[f'{column}_klasse_{group}'] = pd.cut(group_df[column], bins=[-np.inf, *group_cuts, np.inf], labels=labels)
        return group_df
    
    if group:
        return df.groupby(group).apply(bin_group)
    else:
        return bin_group(df)

# Apply the function to the DataFrame with and without grouping
EigenAnalyses_grouped = bin_scores(EigenAnalyses, group='Community')
EigenAnalyses_grouped2 = bin_scores(EigenAnalyses)

print("Grouped by 'Group':")
print(EigenAnalyses_grouped.head)
print(EigenAnalyses_grouped2.head)

Grouped by 'Group':
<bound method NDFrame.head of                          NiNID      PC_small      PC_large           REv  \
Community                                                                  
1         0    NINFP2110013705  4.997741e-24  1.234119e-15  2.235468e-12   
          2    NINFP2110029289  1.526052e-24  0.000000e+00  1.235333e-12   
          7    NINFP2110061632  6.310550e-21 -1.234119e-15  7.943885e-11   
          11   NINFP2110061968  2.181544e-24 -3.878660e-15  1.477015e-12   
          13   NINFP2110017878  7.139731e-25 -2.644541e-15  8.450128e-13   
...                        ...           ...           ...           ...   
19        491  NINFP1910017345  1.226786e-02  1.240306e-02  1.107604e-01   
          492  NINFP1910017345  1.226786e-02  1.240306e-02  1.107604e-01   
          493  NINFP1910018441  3.473221e-11  3.472441e-11  5.893404e-06   
          497  NINFP1910018413  8.198483e-03  8.029367e-03  9.054547e-02   
          501  NINFP2110059634  1.2267

In [40]:
Gr = "NiNID"
idx = Coms.shape[1]
Coms2 = pd.DataFrame(np.column_stack((np.array(EigenMeasures)[:,0],
                        Coms[:,idx-1])))
Coms2.columns = [Gr,"Community"]
EigenAnalyses = pd.merge(EigenMeasures,Coms2, on = Gr)
EigenAnalyses['PC_large_Area'] = EigenAnalyses['PC_large']/EigenAnalyses['Area']
# Define thresholds and labels
cuts = np.quantile(EigenAnalyses['PC_large'], [1/3, 2/3])
cuts2 = np.quantile(EigenAnalyses['PC_small'], [1/3, 2/3])

labels = ['Lav', 'Middels', 'Høy']
# Bin the scores and write the result to a new column
EigenAnalyses['PC_large_klasse'] = pd.cut(EigenAnalyses['PC_large'], bins=[-np.inf, *cuts, np.inf], labels=labels)
#EigenAnalyses['PC_small_klasse'] = pd.cut(EigenAnalyses['PC_small'], bins=[-np.inf, *cuts2, np.inf], labels=labels)

print(EigenAnalyses.head)

<bound method NDFrame.head of                NiNID      PC_small      PC_large           REv           LEv  \
0    NINFP2110013705  4.997741e-24  1.234119e-15  2.235468e-12  2.235658e-12   
1    NINFP2010006340  1.169922e-23 -3.173449e-15  3.420397e-12  3.420427e-12   
2    NINFP2110029289  1.526052e-24  0.000000e+00  1.235333e-12  1.235337e-12   
3    NINFP2010014883  2.101455e-34 -3.349752e-15  3.744973e-17  5.611402e-18   
4    NINFP1810016775  1.790253e-35 -1.586725e-15  1.850446e-17  9.674714e-19   
..               ...           ...           ...           ...           ...   
497  NINFP1910018413  8.198483e-03  8.029367e-03  9.054547e-02  9.054547e-02   
498  NINFP1910010840  8.391615e-22 -2.997146e-15  2.896829e-11  2.896827e-11   
499  NINFP2110012433  2.979365e-26 -5.289082e-15  1.725889e-13  1.726279e-13   
500                   1.611879e-40 -1.763027e-15  4.584858e-23  3.515657e-18   
501  NINFP2110059634  1.226786e-02  1.240306e-02  1.107604e-01  1.107604e-01   

         

Write results back to file geodatabase etc. 

## Beregne sensitiviteter

In [41]:
sensMatrix = np.outer(EigenAnalyses["REv"][0:40],EigenAnalyses["REv"][0:40])
sensDF = pd.DataFrame(sensMatrix, index = EigenAnalyses["NiNID"][0:40], columns = EigenAnalyses["NiNID"][0:40])
sensDF = pd.melt(sensDF.reset_index(), id_vars=['NiNID'], value_vars=EigenAnalyses["NiNID"][0:40], 
                    var_name='NiNID_2', value_name='Sensitivity')

# Define thresholds and labels
cuts = np.quantile(sensDF['Sensitivity'], [1/3, 2/3])
labels = ['Lav', 'Middels', 'Høy']

# Bin the scores and write the result to a new column
sensDF['Sensitivity_klasse'] = pd.cut(sensDF['Sensitivity'], bins=[-np.inf, *cuts, np.inf], labels=labels)

tmpTBL = EigenAnalyses[['NiNID', "Community"]]
# Merge Group from Table 1 into Table 2 based on ID1
sensDF2 = sensDF.merge(tmpTBL, left_on='NiNID', right_on='NiNID', how='left').rename(columns={'Community': 'Community_1'})#.drop(columns=['NiNID'])
#print(sensDF2.merge(tmpTBL, left_on='NiNID_2', right_on='NiNID', how='left'))
# Merge Group from Table 1 into Table 2 based on ID2
sensDF2 = sensDF2.merge(tmpTBL, left_on='NiNID_2', right_on='NiNID', how='left').rename(columns={'Community': 'Community_2','NiNID_x': 'NiNID'}).drop(columns=['NiNID_y'])

subSens = sensDF2[sensDF2['Sensitivity_klasse']=='Høy']
print(subSens)


                NiNID          NiNID_2   Sensitivity Sensitivity_klasse  \
5     NINFP1910019514  NINFP2110013705  2.256702e-13                Høy   
10    NINFP1910019576  NINFP2110013705  9.281158e-14                Høy   
14    NINFP1910019570  NINFP2110013705  3.164192e-13                Høy   
33    NINFP1910017755  NINFP2110013705  3.915374e-14                Høy   
35    NINFP1910019478  NINFP2110013705  2.809535e-13                Høy   
...               ...              ...           ...                ...   
2687  NINFP1910019478  NINFP2010048283  6.124512e-13                Høy   
2690  NINFP2010006408  NINFP2010048283  4.262892e-21                Høy   
2695  NINFP2010058626  NINFP2010048283  1.094992e-19                Høy   
2697  NINFP1910019588  NINFP2010048283  3.351486e-13                Høy   
2701  NINFP1910019566  NINFP2010048283  4.349292e-13                Høy   

     Community_1 Community_2  
5             19           1  
10            19           1  
14    

### LEGG TIL EIGEN-MÅLENE TIL GEOMETRIE

In [50]:
print(list(EigenAnalyses.columns)[1:])
print(list(EigenAnalyses.dtypes[1:]))
print(EigenAnalyses.dtypes.items)

['PC_small', 'PC_large', 'REv', 'LEv', 'Area', 'Community', 'PC_large_Area', 'PC_large_klasse', 'PC_small_klasse']
[dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('O'), dtype('float64'), CategoricalDtype(categories=['Lav', 'Middels', 'Høy'], ordered=True), CategoricalDtype(categories=['Lav', 'Middels', 'Høy'], ordered=True)]
<bound method Series.items of NiNID                object
PC_small            float64
PC_large            float64
REv                 float64
LEv                 float64
Area                float64
Community            object
PC_large_Area       float64
PC_large_klasse    category
PC_small_klasse    category
dtype: object>


In [51]:
# Write the measure to the input_fc
# Add fields for 
# Add a new field to the feature class for each column in the DataFrame
#fields_to_add = ["PC_small", "PC_large", "REv", "LEv","Community"]
# Read all columns except the first one, which is NiNID
fields_to_add = list(EigenAnalyses.columns)[1:]

# Map pandas dtypes to ArcPy field types
dtype_mapping = {'int64': 'LONG', 'float64': 'DOUBLE', 'object': 'TEXT','category': 'TEXT'}

# Add fields to the feature class based on DataFrame column data types
for field, dtype in EigenAnalyses.dtypes.items():
    if field != 'ID':  # Skip the ID column
        arcpy.management.AddField(input_fc, field, dtype_mapping[str(dtype)])


# Define the fields to use in the cursor, including the OID field
fields = ['NiNID'] + fields_to_add

# Use an update cursor to iterate through the feature class
with arcpy.da.UpdateCursor(input_fc, fields) as cursor:
    for row in cursor:
        oid_value = row[0]  # Get the OID value from the feature class
        # Find the corresponding row in the DataFrame based on the OID value
        df_row = EigenAnalyses.loc[EigenAnalyses['NiNID'] == oid_value]
        if not df_row.empty:
            # Update the fields in the feature class with values from the DataFrame
            for i, field in enumerate(fields_to_add):
                row[i + 1] = df_row[field].values[0]  # Skip the first field (OID)
            cursor.updateRow(row)
        else:
            print(f"No matching row found for OID {oid_value}")            

print("Finished")

Finished


In [53]:
print(len(areas[0]))
#plt.hist(EigenAnalyses['Area'])
#plt.scatter(1,areas[0])
#plt.scatter(EigenAnalyses['PC_large']/EigenAnalyses['Area'],EigenAnalyses['PC_small'])
#plt.show()


460


# Lag koblingene som egen fil med sensitivetsmål

Neste steg vil være at en tar utgangspunkt i sensitivitetsmatrisen. Henter ut f.eks. de mest verdifulle lenkene, og lager en linjegeometri basert på denne. Det vil da være av interesse å skille på lenker som går mellom og innad klynger. 

In [54]:
# Convert the input to points, for better visualization
arcpy.FeatureToPoint_management(input_fc, out_folder_path+"/"+gdb_name+"/Pts"
, "INSIDE")

<Result 'C:\\Users\\endofs\\OneDrive - Miljødirektoratet\\Kartfiler\\Nettverk\\Nettverk.gdb\\Pts'>

### Add the points among which the sensitivities are highest, as lines

In [55]:
# List of IDs to subset
print("H")
# Define the output feature class
output_fc = out_folder_path+"//"+gdb_name
print(output_fc)
id_list = list(subSens["NiNID"])+list(subSens["NiNID"])

# Create a SQL query to select the IDs
id_string = ', '.join(map(str, id_list))
sql_query = '"NiNID" in ({:s})'.format(','.join(["'" + str(x) +"'" for x in id_list]))
# Create a feature layer with the subset of points
print("Query")
subset_layer = "subset_layer"
#arcpy.MakeFeatureLayer_management(input_fc, subset_layer, )

arcpy.MakeFeatureLayer_management(input_fc, out_folder_path+"/"+gdb_name+"/subset_layer", sql_query)


H
C:/Users/endofs/OneDrive - Miljødirektoratet/Kartfiler/Nettverk//Nettverk.gdb
Query


<Result 'C:/Users/endofs/OneDrive - Miljødirektoratet/Kartfiler/Nettverk/Nettverk.gdb/subset_layer'>

FOrsøk på kode hvor vi legger til linjer baser på hvorvidt de er i subsens df eller ikke

In [56]:
import arcpy

# Set the workspace (update this path to your actual workspace)
arcpy.env.workspace = out_folder_path + "/" + gdb_name

# Define the input feature class
tmp_fc = out_folder_path + "/" + gdb_name + "/subset_layer"

# Sample DataFrame (replace with your actual DataFrame)
subSens.loc[:, 'Dyad'] = subSens.apply(lambda row: f"{min(row['NiNID'], row['NiNID_2'])}_{max(row['NiNID'], row['NiNID_2'])}", axis=1)

# Get the set of Dyads from the DataFrame
df_dyads = set(subSens['Dyad'])

# Create an output feature class for the lines
tmp_output_fc = "Linker"
arcpy.CreateFeatureclass_management(arcpy.env.workspace, tmp_output_fc, "POLYLINE")

# Add fields to the output feature class
arcpy.AddField_management(tmp_output_fc, "NiNID_1", "TEXT")
arcpy.AddField_management(tmp_output_fc, "NiNID_2", "TEXT")
arcpy.AddField_management(tmp_output_fc, "Dyad", "TEXT")

# Use a search cursor to iterate through the polygons
polygons = []
with arcpy.da.SearchCursor(tmp_fc, ["SHAPE@", "NiNID"]) as cursor:
    for row in cursor:
        polygons.append((row[0], row[1]))

# Use an insert cursor to create lines between polygons
with arcpy.da.InsertCursor(tmp_output_fc, ["SHAPE@", "NiNID_1", "NiNID_2", "Dyad"]) as cursor:
    for i in range(len(polygons)):
        for j in range(i + 1, len(polygons)):
            start_polygon, start_ninid = polygons[i]
            end_polygon, end_ninid = polygons[j]
            start_point = start_polygon.centroid
            end_point = end_polygon.centroid
            dyad = f"{min(start_ninid, end_ninid)}_{max(start_ninid, end_ninid)}"
            if dyad in df_dyads:
                line = arcpy.Polyline(arcpy.Array([start_point, end_point]))
                cursor.insertRow([line, start_ninid, end_ninid, dyad])

print("Dyad field updated, rows deleted, and lines created successfully!")

[56]:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Dyad field updated, rows deleted, and lines created successfully!


OLD CODE